In [1]:
import gymnasium as gym
import numpy as np

try:
     del gym.envs.registry['lungs-v0']
except:
     pass

gym.envs.register(
     id='lungs-v0',
     entry_point='simulator.envs.lung:LungEnv',
     max_episode_steps=250,
)

G = 0.5
LARGE_MAZE_DIVERSE_G = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                        [1, G, 0, 0, 0, 1, G, 0, 0, 0, 0, 1],
                        [1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1],
                        [1, 0, 0, 0, 0, G, 0, 1, 0, 0, G, 1],
                        [1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1],
                        [1, 0, G, 1, 0, 1, 0, 0, 0, 0, 0, 1],
                        [1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1],
                        [1, 0, 0, 1, G, 0, G, 1, 0, G, 0, 1],
                        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]



env = gym.make('lungs-v0', maze_map = LARGE_MAZE_DIVERSE_G, max_episode_steps=250)


In [2]:
# Box(4,) means that it is a Vector with 4 components
print("Observation space:", env.observation_space)
# Discrete(2) means that there is two discrete actions
print("Action space:", env.action_space)

Observation space: Dict('achieved_goal': Box(-inf, inf, (2,), float64), 'desired_goal': Box(-inf, inf, (2,), float64), 'maze_map': Box(-inf, inf, (108,), float64), 'observation': Box(-inf, inf, (4,), float64))
Action space: Box(-1.0, 1.0, (2,), float32)


# Train

In [3]:
from stable_baselines3 import PPO

model = PPO("MultiInputPolicy", env, verbose=0, device='cpu')
model.learn(total_timesteps=100, progress_bar=True)


# vec_env = model.get_env()
# obs = vec_env.reset()
# for i in range(10):
#     action, _states = model.predict(obs, deterministic=True)
#     obs, reward, done, info = vec_env.step(action)
#     vec_env.render()
#     # VecEnv resets automatically
#     # if done:
#     #   obs = env.reset()

env.close()

Output()

/home/flaccagora/.miniconda3/envs/RL/lib/python3.9/site-packages/stable_baselines3/common/vec_env/base_vec_env.py:243: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


# Save and/or Test

In [4]:
model.save("models/ppo_lungs_1Mbis")
del model
del env

In [7]:
from stable_baselines3 import PPO

model = PPO.load("models/ppo_lungs_1M", device='cpu')
env = gym.make('lungs-v0',render_mode ='human', maze_map = LARGE_MAZE_DIVERSE_G, max_episode_steps=250)

In [9]:
observation, info = env.reset(seed=412)

terminated = False
truncated = False


while True:
    while not terminated and not truncated:
        #action = policy(observation)  # custom policy
        #action = env.action_space.sample()  #
        action, _states = model.predict(observation, deterministic=True)
        
        observation, reward, terminated, truncated, info = env.step(action)

    observation, info = env.reset()
    terminated = False
    truncated = False

env.close()